# SLA Investigation
1. Run all cells.
1. View report at the bottom.

In [ ]:
triggerTime='2019-07-25 04:01:00.0Z'
scaleUnit=''
service='tfs'

In [ ]:
# bootstrapping
!pip install --upgrade pip azure-kusto-notebooks
import os
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
pd.options.display.html.table_schema = True
import concurrent.futures
from azure.kusto.notebooks import utils as akn

In [ ]:
# authenticate kusto client
client = akn.get_client()

In [ ]:
# find orchestrations that violate SLA
import datetime
params = {
    'TriggerTime': akn.to_kusto_datetime(triggerTime),
    'Service': '"' + service + '"', 
    'ScaleUnit': '"' + scaleUnit + '"'
}
violations = akn.execute_file(client, database='VSO', path=os.path.join(repo_path, 'SLADurationAnalysis.csl'), params=params)
# violations

In [ ]:
# collect problematic orchestration ids
result = violations.primary_results[0]
oid_column_index = next((c.ordinal for c in result.columns if c.column_name == 'OrchestrationId'), None)
su_column_index = next((c.ordinal for c in result.columns if c.column_name == 'ScaleUnit'), None)

# group
by_su = {}
for r in result.rows:
    su = r[su_column_index]
    oid = r[oid_column_index]
    l = by_su.get(su, [])
    by_su[su] = l
    l.append(oid)
number_of_violations = len(result.rows)

max_scale_units = []
max_problems = 0
for k,v in by_su.items():
  c = len(v)
  if c > max_problems:
    max_problems = c
    max_scale_units = [k]
  elif c == max_problems:
    max_scale_units.append(k)
max_scale_units.sort()

# for su, oids in by_su.items():
#     print(su)
#     for oid in oids:
#         print('   ', oid)

In [ ]:
# collect histories for each bad plan
with concurrent.futures.ThreadPoolExecutor() as executor:
    hfs = [executor.submit(akn.execute_file, client, 'VSO', os.path.join(repo_path, 'SLAVisualization.csl'), 
            {
                'ScaleUnit': '"' + r[su_column_index] + '"', 
                'OrchestrationId': '"' + r[oid_column_index] + '"'
            }) for r in result.rows]
    histories = [h.result() for h in concurrent.futures.as_completed(hfs)]

# convert to data frames
primary_results = [h.primary_results[0] for h in histories]
with concurrent.futures.ThreadPoolExecutor() as executor:
    dataframe_futures = [executor.submit(akn.to_dataframe, r) for r in primary_results]
    dataframes = [dff.result() for dff in concurrent.futures.as_completed(dataframe_futures)]
# histories = None

In [ ]:
# what was the worst phase?
combined = pd.concat(dataframes, ignore_index=True)
worst_df = combined.loc[combined['Level'] == 2].groupby(['PhaseName']).size().to_frame('Count').nlargest(1, 'Count')
worst_phaseName = worst_df.index[0]
worst_count = worst_df.iat[0, 0]
worst_team = worst_phaseName.split('.')[0]

In [ ]:
print('INSIGHT! There are', len(violations.primary_results[0].rows), 'plans out of SLA.')
print('INSIGHT: the most problems (' + str(max_problems) + ')', 'are in', ', '.join(max_scale_units))
print('INSIGHT: There might be a problem with', worst_phaseName + '.', 
      'It was the slowest in', worst_count, 'of the', number_of_violations, 'SLA violations.')
print('ACTION: open icm against scale units:', max_scale_units, ', assign it to:', worst_team)

In [ ]:
# view all histories
%matplotlib inline

plt.rcdefaults()
fig, axes = plt.subplots(nrows=number_of_violations, 
                         ncols=1, 
                         figsize=(8, 6 * number_of_violations),
                         constrained_layout=True)

vdf = akn.to_dataframe(violations.primary_results[0])
for i in range(number_of_violations):
    df = dataframes[i][['PlanId', 'PhaseName', 'PercentDifference']]
    ax = axes[i] if number_of_violations > 1 else axes
    ax.axhline(0, color='k')
    
    x = df['PhaseName']
    xpos = np.arange(len(x))
    y = df['PercentDifference']
    plan_id = df['PlanId'].iloc[0]
    total_duration = 'duration: ' + str(vdf.loc[vdf['PlanId'] == plan_id]['PlanDuration'].iloc[0])
    ax.title.set_text('\n'.join([plan_id, total_duration]))
    
    ax.bar(x=xpos, height=y)
    ax.set_xticks(xpos)
    ax.set_xticklabels(x, rotation=45, ha="right")

    
# output_filename = 'analysis.svg'
# plt.savefig(output_filename, format='svg')